# Training and evaluating Singing Voice Detectors

In [2]:
import sys
sys.path.append('../vocaldetection/')
import sklearn
import utils
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import librosa
import arff, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

I know that the best parameters for training SVM and RF on this dataset are C=0.1 for SVM and nE=100 for RF. [WHY??]

Then, I can just train the models with these parameters and see how the results on confusion changes.

For each set of descriptors, I can do a cross validation evaluation and see the accuracy for each classifier.

Then, I select the greater accuracy to be my reference of ceiling confusion.

In [3]:
features = ['MFCC','VV','FL','SC','SF']

In [4]:
labels = ['absent','present']

In [9]:
# Path for features calculated with Lehner Code
all_feat_path = '/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/'
f = all_feat_path+'ICASSP2014RNN/'

mfcc_path = all_feat_path+'MFCC_29_30_0_0.5_0dt/40_20_40/'
# Read features and labels
FEAT_PATH = os.environ["FEAT_PATH"]
AUDIO_PATH = os.environ["AUDIO_PATH"]
PIECES = os.environ["PIECES_JSON"]

### Calculate Vocal Variance Separated

In [6]:
# # vocal variance parameters 
# FRAMESIZE = 2205
# HOPSIZE = 441
# SR = 22050
# # Function taken from https://github.com/kyungyunlee/ismir2018-revisiting-svd/blob/master/lehner_randomforest/vocal_var.py
# def vocal_var(audiofile):
#     ''' Compute vocal variance
#     '''
#     mfcc = librosa.feature.mfcc(audiofile, sr=SR, n_mfcc=30, n_fft=FRAMESIZE, hop_length=HOPSIZE, n_mels=128)
#     mfcc = mfcc.swapaxes(0, 1)
#     vv = np.empty([len(mfcc), 5])
#     for i in range(len(mfcc)):
#         for j in range(5):
#             vv[i][j] = np.var(mfcc[max(0, i - 5): min(len(mfcc), i + 6), j + 1])
#     return vv

In [7]:
music_files = []

with open(PIECES) as json_file:  
    data = json.load(json_file)
    
    for music in data.keys():
        music_files.append(music)

## Read MFCC files

In [11]:
train_features = []
train_labels = []

for tf in music_files:
    try:
        dataset = arff.load(open(mfcc_path+tf+'_MIX.arff', 'r'))
        data = np.array(dataset['data'])
        print (mfcc_path+tf+'_MIX.arff')
    except FileNotFoundError:
        print ('File not found: ',mfcc_path+tf+'_MIX.arff')
        continue

    # Calculate VV because it is not included on Lehner feature pack
    #audiofile, _ = librosa.load(AUDIO_PATH+tf+'/'+tf+'_MIX.wav', sr=SR)
    #vv=lbl = np.load(FEAT_PATH+tf+"_vv_lee.npy")
    #print (vv.shape)
    lbl = np.load(FEAT_PATH+tf+"_labels_20ms.npy")
    
    feature_vector = []
    for idx in range(len(lbl)):
        #feature_vector.append(np.concatenate((data[idx], vv[idx]), axis=0))
        feature_vector.append(data[idx])
        
    # Store the feature vector and corresponding label in integer format
    for idx in range(len(feature_vector)):
        train_features.append(feature_vector[idx])
        train_labels.append(lbl[idx])
        
    print (len(train_features))
    print (len(train_labels))


/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/AClassicEducation_NightOwl_MIX.arff
8572
8572
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/AimeeNorwich_Child_MIX.arff
18035
18035
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/AlexanderRoss_GoodbyeBolero_MIX.arff
38975
38975
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/AlexanderRoss_VelvetCurtain_MIX.arff
64699
64699
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/Auctioneer_OurFutureFaces_MIX.arff
75093
75093
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/AvaLuna_Waterduct_MIX.arff
88058
88058
/media/shayenne/CompMusHD/DATASETS/MedleyDB/AllMusics/ICASSP2014/MFCC_29_30_0_0.5_0dt/40_20_40/BigTroubles_Phantom_MIX.arff
95404
95404
/media/shayenne/CompMusH

In [12]:
X = np.array(train_features)
y = np.array(train_labels)

In [13]:
# Percentage of singing voice frames on dataset
print (sum(y)/len(X))

0.6880463547294686


In [14]:
# Split dataset into train, validation and test sets

In [15]:
from sklearn.model_selection import StratifiedKFold # import KFold
seed = 7
kf = StratifiedKFold(n_splits=10, random_state=seed) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

10

In [16]:
for train_index, test_index in kf.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 38589  38590  38591 ... 608004 608005 608006] TEST: [    0     1     2 ... 71112 71113 71114]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [ 38589  38590  38591 ... 137641 137642 137643]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [ 87993  87994  87995 ... 202195 202196 202197]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [148568 148569 148570 ... 253047 253048 253049]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [198618 198619 198620 ... 307312 307313 307314]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [286024 286025 286026 ... 380753 380754 380755]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [345997 345998 345999 ... 429463 429464 429465]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [405464 405465 405466 ... 491063 491064 491065]
TRAIN: [     0      1      2 ... 608004 608005 608006] TEST: [473225 473226 473227 ... 551288 551289 551290]
TRAIN: [     0      1    

In [17]:
# Fit models

# Evaluate models

In [18]:
def plot_cm(cm):
    fig, ax = plt.subplots()
    ax.imshow(cm, interpolation='nearest', cmap='gray')
    for i, line in enumerate(cm):
        for j, l in enumerate(line):
            ax.text(j, i, l, size=20, color='green')
    ax.set_xticks(range(len(cm)))
    ax.set_xticklabels(labels)
    ax.set_yticks(range(len(cm)))
    ax.set_yticklabels(labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    plt.show()

In [19]:
X.shape

(608007, 60)

### Random Forest Classifier without scaler

In [ ]:
# Use scikit-learn to train a model with the training features we've extracted
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest

def rf_param_selection(X, y, nfolds):
    n_estimators = [int(x) for x in np.linspace(10, 200, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]# Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    print (random_grid)
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                                   n_iter = 100, cv = nfolds, verbose=2, random_state=42, 
                                   n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X, y)
    print (rf_random.best_params_)
    return rf_random.best_estimator_


In [ ]:
nfolds = 10
best_rf = rf_param_selection(X, y, nfolds)

In [ ]:
rf = RandomForestClassifier(n_estimators=157, min_samples_split=10, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=20, bootstrap=True, random_state=42)
rf.fit(X, y)

In [ ]:
# Now lets predict the labels of the test data!
predictions = rf.predict(X)

In [ ]:
# Biased Guess Accuracy
ones = np.ones(len(predictions))
accuracy = sklearn.metrics.accuracy_score(y, ones)
print(accuracy)

In [ ]:
# RF Accuracy
accuracy = sklearn.metrics.accuracy_score(y, predictions)
print(accuracy)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

metrics = precision_recall_fscore_support(y, predictions)
print('Negatives : ', metrics[3][0],'- Positives',metrics[3][1])
print('Precision :', round(metrics[0][1],3))
print('Recall    :', round(metrics[1][1],3))
print('F-score   :', round(metrics[2][1],3))

In [ ]:
# lets compute the show the confusion matrix:
cm = sklearn.metrics.confusion_matrix(y, predictions)
plot_cm(cm)

# Testing in one piece

In [ ]:
piece_name = 'LizNelson_Rainfall'

In [ ]:
#vggish = pd.read_csv(VGGish_PATH+piece_name+"_VGGish_PCA.csv",index_col=None, header=None)
#vggish = vggish.values
mfcc_piece = mfcc_path+piece_name+'_MIX.arff'
dataset = arff.load(open(mfcc_piece, 'r'))
data = np.array(dataset['data'])

In [ ]:
piece_features = data

In [ ]:
pred_labels = rf.predict(piece_features)

In [ ]:
piece_test = np.load(FEAT_PATH+piece_name+"_labels_20ms.npy")
print (piece_test.shape, piece_features.shape)
#piece_test = piece_test[:piece_features.shape[0]]

In [ ]:
# RF VGGish Accuracy
accuracy = sklearn.metrics.accuracy_score(piece_test, pred_labels)
print(accuracy)

In [ ]:
# lets compute the show the confusion matrix:
cm = sklearn.metrics.confusion_matrix(piece_test, pred_labels)
plot_cm(cm)

In [ ]:
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
plt.figure(figsize=(14,3))
plt.plot(piece_test+2)
plt.plot(pred_labels+1)
plt.plot(scipy.signal.medfilt(pred_labels,kernel_size=5))
plt.plot(proba-1)
plt.plot(scipy.signal.medfilt(proba,kernel_size=5)-2)
thresh = np.zeros(len(proba))
thresh[scipy.signal.medfilt(proba,kernel_size=3) >= 0.5] = 1
plt.plot(thresh-3)
plt.legend(['Original Label', 'Predicted Label', 'Filter Prediction', 
            'Probabilities', 'Filter Probabilities', 'Threshold Prob'], bbox_to_anchor=(1, 1))